In [ ]:
pip install --upgrade opencv-python


In [ ]:
pip install opencv-python dlib


In [ ]:
pip install face_recognition


In [ ]:
import cv2
import face_recognition
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import concurrent.futures

# Load known faces
known_image = face_recognition.load_image_file("WhatsApp Image 2024-07-24 at 17.32.13_ec8e695d.jpg") # trained with my image
known_encoding = face_recognition.face_encodings(known_image)[0]
known_faces = {"Indhu C N": known_encoding}

# Email configuration
SENDER_EMAIL = "sender_mail address"  # replace with sender mail id
RECEIVER_EMAIL = "receiver_mail address" # replace with receiver mail id
SENDER_PASSWORD = "xxxxxxxxxx" #replace with ur password
SMTP_SERVER = 'smtp.gmail.com' # replace with ur's
SMTP_PORT = 587


def send_email_alert(image_path):
    msg = MIMEMultipart()
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECEIVER_EMAIL
    msg['Subject'] = 'Unknown Person Detected'

    body = 'An unknown person was detected by your security camera.'
    msg.attach(MIMEText(body, 'plain'))

    # Attach image
    with open(image_path, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(image_path)}')
        msg.attach(part)

    # Send email
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())

    os.remove(image_path)

def async_send_email(image_path):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(send_email_alert, image_path)
        return future

# Initialize webcam
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Could not open video capture.")
    exit()

frame_skip = 2  # Number of frames to skip
frame_count = 0
scale_factor = 0.5  # Reduce to half size

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    frame_count += 1

    # Check if frame was successfully captured
    if not ret:
        print("Error: Could not read frame.")
        break

    # Skip frames to reduce load
    if frame_count % frame_skip != 0:
        continue

    # Resize the frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert the image from BGR color to RGB color
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Scale back up face locations since the frame we detected in was scaled down
        top *= int(1 / scale_factor)
        right *= int(1 / scale_factor)
        bottom *= int(1 / scale_factor)
        left *= int(1 / scale_factor)

        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(list(known_faces.values()), face_encoding)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = list(known_faces.keys())[first_match_index]
        else:
            # Save the unknown person's image
            unknown_face = frame[top:bottom, left:right]
            image_path = f'unknown_{top}_{left}.jpg'
            cv2.imwrite(image_path, unknown_face)

            # Send alert if unknown face detected
            future = async_send_email(image_path)
            future.result()  # Wait for email sending to complete if necessary

        # Draw rectangle around the face
        color = (0, 0, 255) if name == "Unknown" else (0, 255, 0)
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

        # Display the name on the screen
        cv2.putText(frame, name, (left, bottom + 20), cv2.FONT_HERSHEY_COMPLEX, 0.75, color, 2)
        print(f"Detected: {name}")

    # Display the resulting image
    cv2.imshow('Security Camera Visual', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
video_capture.release()
cv2.destroyAllWindows()
